In [15]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
import json, torch, bitsandbytes, gTTS

ModuleNotFoundError: No module named 'gTTS'

In [2]:
# 경로 및 모델명
file_path = "train_data/cleaned_dataset_half.json"
model_name = "beomi/KoAlpaca-Polyglot-5.8B"

# 1. 데이터 로드
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)

In [3]:
# 2. 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # LLaMA 계열에서 필수

# 3. 토크나이징 함수 정의
def tokenize(example):
    prompt = example["input"].strip()
    response = example["output"].strip()
    full_text = prompt + "\n" + response

    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize, remove_columns=["input", "output"])

Map: 100%|██████████| 6353/6353 [00:02<00:00, 2746.42 examples/s]


In [14]:
# 4. 양자화 설정 + 모델 로드
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=False  # CPU offload 금지
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)


CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
# 5. LoRA 설정 및 적용
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["attention.query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)


model = get_peft_model(model, lora_config)


In [ ]:
# 6. 학습 설정
training_args = TrainingArguments(
    output_dir="train_data/koalpaca-lora-checkpoints",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
    report_to="none",
    fp16=True
)

In [ ]:
# 7. 데이터 콜레이터 및 Trainer
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# 8. 학습 실행
trainer.train()

# 9. 모델 저장
trainer.save_model("train_data/beomi_llm")
tokenizer.save_pretrained("train_data/beomi_llm")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization


Step,Training Loss
10,4.707900
20,4.605600
30,4.539400
40,4.251400
50,4.105100
60,3.852100
70,3.490200
80,3.285100
90,3.146900
100,3.109800


('/content/beomi_llm/tokenizer_config.json',
 '/content/beomi_llm/special_tokens_map.json',
 '/content/beomi_llm/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer

base_model = "beomi/KoAlpaca-Polyglot-5.8B"
lora_model_path = "train_data/beomi_llm"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(lora_model_path, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # 필수

# 베이스 모델 로드 + LoRA 적용
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    load_in_8bit=True
)
model = PeftModel.from_pretrained(model, lora_model_path)

model.eval()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 4096)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-27): 28 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (query_key_value): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=4096, out_features=12288, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_feature

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def generate(prompt, max_new_tokens):
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items() if k != "token_type_ids"}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.5,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
import re

def clean_and_trim_response(prompt, generated_text):
    # prompt 이후 부분만 추출
    response = generated_text[len(prompt):].strip()

    # 불필요한 토큰 제거
    response = re.sub(r'(input:|output:)', '', response).strip()

    # 마침표 기준으로 문장 자르기 후 첫 문장만 사용
    sentences = re.split(r'(?<=[.!?])\s+', response)
    return sentences[0] if sentences else response


input_text = "오늘 밥을 뭘 먹는게 좋을까?"
prompt = f"input: {input_text}\noutput:"
generated = generate(prompt, 100)

result = clean_and_trim_response(prompt, generated)
print(result)


오늘은 그냥 간단하게 샌드위치를 먹어볼까?


In [ ]:
# 언어 설정
language = 'ko'

# gTTS 객체 생성
speech = gTTS(text=result, lang=language, slow=False)

# 음성 파일로 저장
speech.save("tts_output_wav")


In [ ]:
import shutil

source_dir = '/content/beomi_llm'
destination_dir = '/content/drive/MyDrive/디스부/예비 프로젝트/Beomi/beomi_llm'

shutil.copytree(source_dir, destination_dir)


'/content/drive/MyDrive/디스부/예비 프로젝트/Beomi/beomi_llm'